In [118]:
import boto3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sagemaker.session import Session
from joblib import dump, load

from sklearn.feature_extraction.text import HashingVectorizer

In [119]:
assetMDL = load('Outs/asset_svc_mdl_v1.joblib')
liableMDL = load('Outs/liability_svc_mdl_v1.joblib')

In [120]:
# load in asset and liability dataframes
assetDF = pd.read_csv('unstructAsset.csv')
liableDF = pd.read_csv('unstructLiable.csv')

In [163]:
# journal of physics A, Journal Stat. Physics
def structured_data(unstructured_df:pd.DataFrame, cluster_df:pd.DataFrame) -> pd.DataFrame:
    """
    Constructs a structured dataset from an unstructured column set
    
    :param: unstructured_df (type pandas.DataFrame)
        unstuructured pandas dataframe with loose column construction 
    :param: cluster_df (type pandas.DataFrame)
        a pandas dataframe of clustered labels and corresponding line items
    :param: (type numpy array)
        all corresponding cluster labels cirresponding with 'cluster_df' parameter
        
    :return: (type pandas DataFrame)
    """
    
    structured_df = pd.DataFrame()
    label_names = np.unique(cluster_df.Labels.values)
    remap = {}
    
    # assume that the there exists columns 'CIK' and 'Year' for unstructured data
    structured_df = unstructured_df[['CIK', 'Name', 'Year']]
    
    for label in label_names:
        data = cluster_df[cluster_df['Labels'] == label]['LineItems']     # filter by corresponding cluster
        
        # we first select all predicted columns, sum across rows, replace 0 with NaN and report figures
        sumV = unstructured_df[data.values].sum(axis=1).replace({0:np.nan})
        
        # assign dictionary to have labels and matching vector
        remap[label] = sumV

    structured_df = structured_df.assign(**remap)   
    return structured_df

## Use Classificaiton model to predict label names for each line item

In [122]:
asset_predictions = pd.DataFrame([assetDF.columns[3:], 
                                  assetMDL.predict(HashingVectorizer(n_features=1000).fit_transform(assetDF.columns[3:]))], 
                                 index=['LineItems', 'Labels']).T

liable_predictions = pd.DataFrame([liableDF.columns[3:], 
                                   liableMDL.predict(HashingVectorizer(n_features=1000).fit_transform(liableDF.columns[3:]))], 
                                  index=['LineItems', 'Labels']).T

## Examine robustness of structured databases

In [175]:
assetterms = ['Account receivable', 'Accumulated depreciation and amortization', 'Cash and cash equivalents',
              'Cash and securities segregated for benefit of customers', 'Deferred tax asset', 
              'Deposits with clearing organizations', 'Due from broker dealer', 'Due from customer and counterparties',
              'Due from employees', 'Due from parent and affiliates', 'Escrow account balances', 'Exchange memberships',
              'Financial instruments owned, at fair value', 'Fixed assets','Goodwill, net amortization', 
              'Other assets', 'Other receivables', 'Prepaid expenses and other assets', 'Property, plant and equipment', 
              'Receivable from broker-dealers', 'Receivables from customers and counterparties',
              'Reverse Repurchase Agreements (reverse-repo)', 'Securities received as collateral', 
              'U.S. government and government agency']
liableterms = ['Accounts payable', 'Accrued liabilities', 'Current liabilities', 
               'Deferred liability', 'Due to customers and members', 'Due to parent and affiliates',
               'Due to third party affiliates', 'Government and agency securities obligations', 'Income tax payable', 
               'Lease Liability', 'Long-term borrowing', 'Other liabilities', 'Payable to Broker/Dealers',
               'Payable to customers and counterparties', 'Payable to parent and affiliates', 
               'Repurchase Agreements (repo)', 'Securities borrowed', 'Securities sold short', 'Short-term borrowing',
               'Subordinated liabilities', 'Taxes payable']
equityterms = ['Common stock, par value', 'Other equity', 'Retained (Accumulated) earnings', 'Treasury stock',
               'Additional Paid-in capital',]

In [171]:
# construct the strucutred data set 
tempdf = structured_data(assetDF, asset_predictions)
tempdf.to_csv('structAsset.csv', index=False)

# filter out the Total Assets total 
cols = tempdf.columns[3:]
cols = cols[~np.isin(cols, 'Total assets')]

# compute the total asset check for each of the firms (robustness for equality figure)
tempdf = tempdf.assign(Check=(tempdf['Total assets'] == tempdf[cols].sum(axis=1)))

In [172]:
tempdf[tempdf.Check == False]

,CIK,Name,Year,Accumulated depreciation and amortization,Cash and cash equivalents,Cash and securities segregated for benefit of customers,Deposits with clearing organizations,Exchange memberships,"Financial instruments owned, at fair value",Fixed assets,...,Other assets,Other receivables,"Property, plant and equipment",Receivable from broker-dealers,Receivables from customers and counterparties,Reverse Repurchase Agreements (reverse-repo),Securities received as collateral,Total assets,U.S. government and government agency,Check
17,42352,GOLDMAN SACHS & CO. LLC,2002,NaN,24258610.0,NaN,NaN,NaN,31630471.0,NaN,...,1941781.0,NaN,NaN,4408164.0,13165513.0,114571529.0,NaN,NaN,NaN,False
18,42352,GOLDMAN SACHS & CO. LLC,2003,NaN,19384642.0,NaN,NaN,NaN,81056494.0,NaN,...,1716894.0,NaN,NaN,3964833.0,10265518.0,138122674.0,NaN,NaN,NaN,False
19,42352,GOLDMAN SACHS & CO. LLC,2004,NaN,26573659.0,NaN,NaN,NaN,106642938.0,NaN,...,1981583.0,NaN,NaN,7976880.0,10289117.0,167604597.0,NaN,NaN,NaN,False
20,42352,GOLDMAN SACHS & CO. LLC,2005,NaN,34330737.0,NaN,NaN,NaN,130905560.0,NaN,...,3329362.0,NaN,NaN,8974752.0,12342912.0,224160434.0,NaN,348590977.0,NaN,False
21,42352,GOLDMAN SACHS & CO. LLC,2006,NaN,43053739.0,NaN,NaN,NaN,212709782.0,NaN,...,5357602.0,NaN,NaN,9648246.0,14642116.0,296815545.0,NaN,475872139.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,91154,CITIGROUP GLOBAL MARKETS INC.,2016,NaN,3641.0,NaN,NaN,NaN,NaN,NaN,...,62439.0,NaN,279.0,13239.0,48702.0,160671.0,8716.0,253416.0,24756.0,False
155,91154,CITIGROUP GLOBAL MARKETS INC.,2017,NaN,5386.0,NaN,NaN,NaN,NaN,NaN,...,66032.0,NaN,71.0,13446.0,46941.0,160143.0,9307.0,261644.0,36821.0,False
156,91154,CITIGROUP GLOBAL MARKETS INC.,2018,NaN,5146.0,NaN,NaN,NaN,NaN,NaN,...,62383.0,NaN,NaN,11419.0,51904.0,162683.0,15443.0,272544.0,31492.0,False
157,91154,CITIGROUP GLOBAL MARKETS INC.,2019,NaN,5875.0,NaN,NaN,NaN,NaN,NaN,...,73741.0,NaN,NaN,11405.0,54181.0,173896.0,15877.0,298908.0,37088.0,False


In [176]:
# construct the strucutred data set 
tempdf = structured_data(liableDF, liable_predictions)
tempdf.to_csv('structLiable.csv', index=False)

# filter out the Total Liability, Equity and Combined totals  
cols = tempdf.columns[3:]
cols = cols[~np.isin(cols, ['Total liabilities', "Total liabilities and shareholder's equity", 
                            "Total shareholder's equity"])]

# compute the total asset check for each of the firms (robustness for equality figure)
tempdf = tempdf.assign(Check1=(tempdf['Total liabilities'] == tempdf[liableterms].sum(axis=1)))
tempdf = tempdf.assign(Check2=(tempdf["Total shareholder's equity"] == tempdf[equityterms].sum(axis=1)))
tempdf = tempdf.assign(Check2=(tempdf["Total liabilities and shareholder's equity"] == tempdf[cols].sum(axis=1)))

KeyError: "['Lease Liability', 'Deferred liability', 'Due to third party affiliates', 'Due to parent and affiliates', 'Taxes payable', 'Due to customers and members'] not in index"

In [177]:
tempdf 

,CIK,Name,Year,Accounts payable,Accrued liabilities,Additional Paid-in capital,"Common stock, par value",Current liabilities,Government and agency securities obligations,Income tax payable,...,Repurchase Agreements (repo),Retained (Accumulated) earnings,Securities borrowed,Securities sold short,Short-term borrowing,Subordinated liabilities,Total liabilities,Total liabilities and shareholder's equity,Total shareholder's equity,Treasury stock
0,1224385,"WELLS FARGO SECURITIES, LLC",2004,NaN,27595.0,NaN,NaN,NaN,NaN,NaN,...,13602472.0,39849.0,NaN,3686980.0,NaN,538500.0,19715370.0,21509787.0,1255917.0,NaN
1,1224385,"WELLS FARGO SECURITIES, LLC",2005,NaN,83799.0,NaN,NaN,NaN,NaN,NaN,...,21430507.0,NaN,NaN,6775259.0,NaN,1183500.0,32320448.0,35611655.0,2107707.0,NaN
2,1224385,"WELLS FARGO SECURITIES, LLC",2006,NaN,59321.0,NaN,NaN,NaN,NaN,NaN,...,12026974.0,NaN,NaN,5059406.0,NaN,1183500.0,20118057.0,24029490.0,2727933.0,NaN
3,1224385,"WELLS FARGO SECURITIES, LLC",2007,2536.0,75497.0,NaN,NaN,NaN,NaN,NaN,...,9294056.0,902450.0,NaN,4241568.0,NaN,1183500.0,16498871.0,20879923.0,3197552.0,NaN
4,1224385,"WELLS FARGO SECURITIES, LLC",2008,NaN,92477.0,NaN,NaN,NaN,NaN,NaN,...,11870678.0,724933.0,NaN,3605979.0,NaN,1333500.0,17307282.0,21660817.0,3020035.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,91154,CITIGROUP GLOBAL MARKETS INC.,2016,NaN,NaN,9099.0,10.0,NaN,24756.0,NaN,...,129960.0,33.0,NaN,NaN,10606.0,9945.0,234329.0,253416.0,20415.0,NaN
155,91154,CITIGROUP GLOBAL MARKETS INC.,2017,NaN,NaN,8921.0,10.0,NaN,36821.0,NaN,...,138098.0,263.0,NaN,NaN,5576.0,9945.0,252450.0,261644.0,21067.0,NaN
156,91154,CITIGROUP GLOBAL MARKETS INC.,2018,NaN,NaN,8937.0,10.0,NaN,31492.0,NaN,...,146794.0,68.0,NaN,NaN,1251.0,9945.0,263529.0,272544.0,18984.0,NaN
157,91154,CITIGROUP GLOBAL MARKETS INC.,2019,NaN,NaN,8802.0,10.0,NaN,37088.0,NaN,...,157690.0,48.0,NaN,NaN,508.0,9945.0,290048.0,298908.0,27197.0,NaN


In [126]:
s3 = boto3.client('s3')
session = Session()

In [127]:
# retrieving downloaded files from s3 bucket
s3.download_file("ran-s3-systemic-risk", 'Output/X-17A-5-BS/1224385-2004.csv', 'temp.pdf')
df = pd.read_csv('temp.pdf')

In [128]:
pd.DataFrame([df.iloc[:11]['1'], df.iloc[:11]['2'],
             assetMDL.predict(HashingVectorizer(n_features=1000).fit_transform(df.iloc[:11]['1']))], 
             index=['LineItems', 'Values', 'Labels']).T

,LineItems,Values,Labels
0,Cash,$ 176,Cash and cash equivalents
1,Cash segregated pursuant to federal regulations,"75,000",Cash and cash equivalents
2,Securities purchased under agreements to resell,"7,944,113",Reverse Repurchase Agreements (reverse-repo)
3,"Securities owned, at market value ($8,769,300 ...","9,075,170",Securities received as collateral
4,Receivable from broker-dealers and clearing or...,"4,049,708",Receivable from broker-dealers
5,Receivable from customers,"51,246",Receivables from customers and counterparties
6,Accrued interest receivable,"44,457",Other receivables
7,"Property, equipment, and leasehold improvement...","11,067","Property, plant and equipment"
8,"Goodwill, net","6,112","Goodwill, net amortization"
9,Other assets,"252,738",Other assets


In [129]:
a.iloc[0]

CIK                                                                             1224385
Name                                                       WELLS FARGO SECURITIES, LLC 
Year                                                                               2004
Accumulated depreciation and amortization                                           NaN
Cash and cash equivalents                                                         75176
Cash and securities segregated for benefit of customers                             NaN
Deposits with clearing organizations                                                NaN
Exchange memberships                                                                NaN
Financial instruments owned, at fair value                                          NaN
Fixed assets                                                                        NaN
Goodwill, net amortization                                                         6112
Other assets                    

In [130]:
b = assetDF[assetDF.columns[3:]].iloc[0]

In [136]:
b[~np.isnan(b)]

Accrued interest receivable                                                44457.0
Cash                                                                         176.0
Cash segregated pursuant to federal regulations                            75000.0
Goodwill, net                                                               6112.0
Other assets                                                              252738.0
Property, equipment, and leasehold improvements, net                       11067.0
Receivable from broker-dealers and clearing organizations                4049708.0
Receivable from customers                                                  51246.0
Securities owned, at market value ($8,769,300 pledged as collateral)     9075170.0
Securities purchased under agreements to resell                          7944113.0
Total assets                                                            21509787.0
Name: 0, dtype: float64